In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
import math

# Paths
image_folder = 'images'
fixation_folder = 'fixations'
output_folder = 'output_images'
output_folder_gray = 'output_images_gray'
ground_truth_images_folder = 'gazefixationsdensitymaps'

# Output folders existence check
os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_folder_gray, exist_ok=True) 

# Initialize empty lists
list_of_fixation_lists = []
image_filenames = []

degree = 1 
radians = math.radians(degree)

# Calculate the tangent value
tan_alpha = math.tan(radians)

R = 1200/325
D =900
screen_height=1920
screen_width=1200

# Iterate through the fixation folder
for fixation_filename in os.listdir(fixation_folder):
    # Check if it's a text file
    if fixation_filename.endswith('.txt'):
        # Get the full path of the fixation file
        fixation_path = os.path.join(fixation_folder, fixation_filename)

        # Read the fixation data from the text file
        fixation_points = []
        with open(fixation_path, 'r') as fixation_file:
            for line in fixation_file:
                x_str, y_str = line.strip().split()
                x = int(x_str)
                y = int(y_str)
                fixation_points.append((x, y))

        # Append the fixation data to the list of lists
        list_of_fixation_lists.append(fixation_points)

        # Extract the corresponding image filename
        for image_filename in os.listdir(image_folder):
            image_filename = os.path.join(image_folder, image_filename)
            image_filenames.append(image_filename)



# Empty list to store the saliency maps
saliency_maps = []

# Loop through each image and its corresponding fixation points
for image_filename, fixation_points in zip(image_filenames, list_of_fixation_lists):
    image_path = f"{image_filename}"
    image = Image.open(image_path)

    # Get the dimensions of the image
    image_width, image_height = image.size

    # Create an empty saliency map for the current image
    saliency_map = np.zeros((image_height, image_width))

    # Calculate the saliency map using fixation points
    for fixation_point in fixation_points:
        if (image_width*screen_height)/screen_height<=screen_width:
            ratio =image_height/screen_height
        else:
            ratio =image_width/screen_width
        sigma =ratio *R *D * tan_alpha
        print(sigma)
            
        x0, y0 = fixation_point
        x, y = np.meshgrid(np.arange(image_width), np.arange(image_height))
        gaussian = np.exp(-((x - x0) ** 2 + (y - y0) ** 2) / (2 * sigma ** 2))
        saliency_map += gaussian

    # Normalize the saliency map
    saliency_map /= np.max(saliency_map)

    # Apply a colormap to the normalized saliency map
    colormap = cv2.applyColorMap((saliency_map * 255).astype(np.uint8), cv2.COLORMAP_JET)

    # Overlay the colorized saliency map on the original image
    alpha = 0.7
    overlay = cv2.addWeighted(np.array(image), alpha, colormap, 1 - alpha, 0)

    # Save the resulting color-mapped image to the output folder
    output_image_path = os.path.join(output_folder, os.path.basename(image_filename))
    cv2.imwrite(output_image_path, overlay)

    # Save the resulting grayscale saliency map to the gray output folder
    output_image_path_gray = os.path.join(output_folder_gray, os.path.basename(image_filename))
    cv2.imwrite(output_image_path_gray, (saliency_map * 255).astype(np.uint8))

    # Display
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(saliency_map, cmap='gray', interpolation='nearest')
    plt.axis('off')
    plt.title("Grayscale Saliency Map")

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Color-Mapped Image")

    plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error

# Im making path here 
ground_truth_folder = 'gazefixationsdensitymaps'
result_folder = 'output_images_gray'

# Create dictionaries to store MSE values for each group
mse_values = {}

# fisrt we need to make a common size for each images ;if not it won't work with the same size 
common_size = (256, 256)  


for filename_gt in os.listdir(ground_truth_folder):
    if filename_gt.endswith('.png'): 
        gt_image_path = os.path.join(ground_truth_folder, filename_gt)

        # since we have difference names for each image names , I consider 3 names are same: to comapare
        prefix_gt = '_'.join(filename_gt.split('_')[:3])

        # Find the matching result image by looking for a filename with the same prefix
        for filename_result in os.listdir(result_folder):
            if filename_result.startswith(prefix_gt):
                result_image_path = os.path.join(result_folder, filename_result)
                gt_image = cv2.imread(gt_image_path, cv2.IMREAD_GRAYSCALE)
                result_image = cv2.imread(result_image_path, cv2.IMREAD_GRAYSCALE)

                if gt_image is not None and result_image is not None:
                    gt_image = cv2.resize(gt_image, common_size)
                    result_image = cv2.resize(result_image, common_size)

                    # Calculate MSE ; I m planing not to calculate PCC and MSE easy and make sence than PCC for me 
                    mse = mean_squared_error(gt_image.flatten(), result_image.flatten())

                    # Print the MSE value for this image
                    print(f"Image: {prefix_gt}, MSE: {mse:.4f}")

                    # Display the ground truth and result images
                    plt.figure(figsize=(8, 4))
                    plt.subplot(1, 2, 1)
                    plt.imshow(gt_image, cmap='gray')
                    plt.title('Ground Truth')

                    plt.subplot(1, 2, 2)
                    plt.imshow(result_image, cmap='gray')
                    plt.title(f'Result (MSE: {mse:.4f})')

                    plt.tight_layout()
                    plt.show()
                else:
                    print(f"Error loading images: {gt_image_path} or {result_image_path}")


In [ ]:
from scipy.stats import pearsonr  # Importat librarues: Import Pearson Correlation Coefficient function

# Create a dictionary to store PCC values for each group
pcc_values = {}

# Define a common size for resizing the images
common_size = (256, 256)

for filename_gt in os.listdir(ground_truth_folder):
    if filename_gt.endswith('.png'):
        gt_image_path = os.path.join(ground_truth_folder, filename_gt)

        # Extract the prefix (first three words) from the ground truth filename
        prefix_gt = '_'.join(filename_gt.split('_')[:3])

        # Find the matching result image by looking for a filename with the same prefix
        for filename_result in os.listdir(result_folder):
            if filename_result.startswith(prefix_gt):
                result_image_path = os.path.join(result_folder, filename_result)
                gt_image = cv2.imread(gt_image_path, cv2.IMREAD_GRAYSCALE)
                result_image = cv2.imread(result_image_path, cv2.IMREAD_GRAYSCALE)

                if gt_image is not None and result_image is not None:
                    gt_image = cv2.resize(gt_image, common_size)
                    result_image = cv2.resize(result_image, common_size)

                    # Calculate PCC instead of MSE
                    pcc, _ = pearsonr(gt_image.flatten(), result_image.flatten())

                    # Print the PCC value for this image
                    print(f"Image: {prefix_gt}, PCC: {pcc:.4f}")

                    # Display the ground truth and result images
                    plt.figure(figsize=(8, 4))
                    plt.subplot(1, 2, 1)
                    plt.imshow(gt_image, cmap='gray')
                    plt.title('Ground Truth')

                    plt.subplot(1, 2, 2)
                    plt.imshow(result_image, cmap='gray')
                    plt.title(f'Result (PCC: {pcc:.4f})')

                    plt.tight_layout()
                    plt.show()
                else:
                    print(f"Error loading images: {gt_image_path} or {result_image_path}")


In [ ]:
# Create an empty list to store all MSE values
all_mse_values = []

# Define a common size for resizing the images
common_size = (256, 256)

# Create a list to store the filenames of ground truth images
gt_filenames = [filename for filename in os.listdir(ground_truth_folder) if filename.endswith('.png')]

# Iterate through the ground truth images
for gt_filename in gt_filenames:
    gt_image_path = os.path.join(ground_truth_folder, gt_filename)

    # Extract the prefix (first three words) from the ground truth filename
    prefix_gt = '_'.join(gt_filename.split('_')[:3])

    # Find the matching result image by looking for a filename with the same prefix
    for result_filename in os.listdir(result_folder):
        if result_filename.startswith(prefix_gt):
            result_image_path = os.path.join(result_folder, result_filename)
            gt_image = cv2.imread(gt_image_path, cv2.IMREAD_GRAYSCALE)
            result_image = cv2.imread(result_image_path, cv2.IMREAD_GRAYSCALE)

            if gt_image is not None and result_image is not None:
                gt_image = cv2.resize(gt_image, common_size)
                result_image = cv2.resize(result_image, common_size)

                # Calculate MSE and append it to the list of all MSE values
                mse = mean_squared_error(gt_image.flatten(), result_image.flatten())
                all_mse_values.append(mse)
            else:
                print(f"Error loading images: {gt_image_path} or {result_image_path}")

# Create a scatter plot of all MSE values
plt.figure(figsize=(8, 6))
plt.scatter(range(len(all_mse_values)), all_mse_values, color='b', alpha=0.7)
plt.xlabel('Image Pair Index')
plt.ylabel('MSE Value')
plt.title('MSE Values Across All Image Pairs')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Create an empty list to store all PCC values
all_pcc_values = []

# Define a common size for resizing the images
common_size = (256, 256)

# Create a list to store the filenames of ground truth images
gt_filenames = [filename for filename in os.listdir(ground_truth_folder) if filename.endswith('.png')]

# Iterate through the ground truth images
for gt_filename in gt_filenames:
    gt_image_path = os.path.join(ground_truth_folder, gt_filename)

    # Extract the prefix (first three words) from the ground truth filename
    prefix_gt = '_'.join(gt_filename.split('_')[:3])

    # Find the matching result image by looking for a filename with the same prefix
    for result_filename in os.listdir(result_folder):
        if result_filename.startswith(prefix_gt):
            result_image_path = os.path.join(result_folder, result_filename)
            gt_image = cv2.imread(gt_image_path, cv2.IMREAD_GRAYSCALE)
            result_image = cv2.imread(result_image_path, cv2.IMREAD_GRAYSCALE)

            if gt_image is not None and result_image is not None:
                gt_image = cv2.resize(gt_image, common_size)
                result_image = cv2.resize(result_image, common_size)

                # Calculate PCC and append it to the list of all PCC values
                pcc, _ = pearsonr(gt_image.flatten(), result_image.flatten())
                all_pcc_values.append(pcc)
            else:
                print(f"Error loading images: {gt_image_path} or {result_image_path}")

# Create a scatter plot of all PCC values
plt.figure(figsize=(8, 6))
plt.scatter(range(len(all_pcc_values)), all_pcc_values, color='b', alpha=0.7)
plt.xlabel('Image Pair Index')
plt.ylabel('PCC Value')
plt.title('PCC Values Across All Image Pairs')
plt.grid(True)
plt.tight_layout()
plt.show()
